## **Homework 1**

by Titipat Achakulvisut

### **problem 1**

(a) Create a data set for a one dimensional linear mapping corrupted by Gaussian noise (i.e. a mapping between visual and proprioceptive sensory input). In particular, create: $y = 2*x + e$; where $x$ is drawn from a uniform random distribution (using unifrnd) between -10 and 10 with 1000 samples, and e is a normally distributed noise vector with mean zero and standard deviation of 1 (use normrnd).

In [183]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [184]:
x = 20*np.random.rand(1000) - 10 # uniform distribution from -10 to 10
e = np.random.normal(0, 1, size=1000)
y = 2*x + e

In [ ]:
plt.plot(x, y, '.', alpha=0.3)
plt.show()

(b) Create a set of RBFs. Place their centers at -12 to 12 at every .5 along the $x$ axis. Set the standard deviation of each RBF to 1.

In [ ]:
def rbf_kernel(x, c, sigma=1.):
    """
    Compute radial basis kernel of given domain x
    k(x, y) = exp(-(1/sigma**2)*||x-c||^2)
    """
    return np.exp(-(1./(2*sigma**2))*(x - c)**2)

def pinv(A):
    """Psuedo inverse of matrix A"""
    return np.linalg.inv(A.T.dot(A)).dot(A.T)

In [ ]:
centers = np.arange(-12, 12.5, 0.5) # center of RBFs
x_kernel = np.vstack([rbf_kernel(x, center) for center in centers]).T # map x to set of RBFs

(c) Use linear regression to find W for the weighting of each RBF and show the predicted values of $y$ for each $x$ value. Make a plot with the original $x$ vs $y$ data and the $x$ vs predicted $y$ superimposed.

In [ ]:
W = np.linalg.pinv(x_kernel).dot(y) # compute W using psuedo inverse
y_hat = x_kernel.dot(W) # predicted y

In [ ]:
plt.plot(x, y, '.b', alpha=0.2)
plt.plot(x, y_hat, '.r', alpha=0.5)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(['original data', 'predicted data'])
plt.show()

(d) Add 50 data points to your $x$ vector, all at $x = 6$. Add a corresponding 50 points to your y vector, according to: $y = 2*x + 10 + e$. (the x used here should correspond to the newly added data points) I.e. displace some of the $y$ values at a particular value of $x$ – as in the paper.

In [ ]:
x_concat = 6*np.ones(50)
y_concat = 2*x_concat + 10 + np.random.randn(len(x_concat))
x_new = np.hstack((x, x_concat))
y_new = np.hstack((y, y_concat))

In [ ]:
plt.plot(x_new, y_new, '.', alpha=0.3)
plt.show()

In [ ]:
centers = np.arange(-12, 12.5, 0.5) # center of RBFs
x_new_kernel = np.vstack([rbf_kernel(x_new, center) for center in centers]).T # map x to set of RBFs

In [ ]:
W = np.linalg.pinv(x_new_kernel).dot(y_new) # compute W using psuedo inverse
y_new_hat = x_new_kernel.dot(W) # predicted y

In [ ]:
plt.plot(x_new, y_new, '.b', alpha=0.2)
plt.plot(x_new, y_new_hat, '.r', alpha=0.5)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(['original data', 'predicted data'])
plt.show()

### **problem 2**

generate 2 classes data (port from given `.m` file to python)

**a. Linear classification, 2 classes**

Your job is to create a classifier which produces the correct class labels for each data point. You’ve been provided with the ‘correct’ class labels in the variable `y` in the mfile. Use a linear classifier/perceptron: `y_hat = sign(wx)`. Note that the offset should be included in the `x` data vector (i.e. add another column of 1’s to your data, as for regression). Use gradient descent on the LMS error to find the best fit weights for this classification. Plot the identified weight vector on the same plot as the raw data, incorporating the offset when you plot the vector (i.e. it shouldn’t go through zero). Interpret the result of your classifier according to the position of this weight vector. Also plot the separation boundary for this classifier (i.e.the line orthogonal to the weight vector).

In [ ]:
y = np.vstack([np.ones((100, 1)), -1*np.ones((100, 1))]).flatten()
X1 = np.hstack((np.random.normal(6, 2, size=(100,1)), np.random.normal(2, 2, size=(100, 1))))
X2 = np.hstack((np.random.normal(2, 3, size=(100,1)), np.random.normal(8, 2, size=(100, 1))))
X = np.vstack((X1, X2))

In [ ]:
plt.scatter(X[(y==1).flatten(), 0], X[(y==1).flatten(), 1], color='r')
plt.scatter(X[(y==-1).flatten(), 0], X[(y==-1).flatten(), 1], color='b')
plt.title('Generated data for problem 2b')
plt.show()

In [ ]:
m = len(X)
X = np.hstack((np.ones((m, 1)), X))

In [ ]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

def sigmoid_grad(z):
    return sigmoid(z)*(1 - sigmoid(z))

def compute_grad(X, y, theta):
    """Compute gradient"""
    m = len(X)
    theta_grad = np.zeros_like(theta)
    y_hat = sigmoid(X.dot(theta))
    for i in range(0, X.shape[1]):
        theta_grad[i] = -2*np.sum((y - y_hat)*sigmoid_grad(y_hat)*X[:, i])
    return theta_grad

def compute_cost(X, y, theta):
    y_hat = sigmoid(X.dot(theta))
    return np.sum(y*np.log(y_hat) + (1-y)*np.log(1 - y_hat))

In [ ]:
# gradient descent
J = [] # history of cost
theta = np.array([1, 1, 1]) # intial theta
n_iter = 2000 # number of iteration
alpha = 0.01 # learning rate
for i in range(n_iter):
    theta_grad = compute_grad(X, y, theta)
    theta = theta - alpha*theta_grad
    #J.append(compute_cost(X, y, theta))
print('final weight theta = ', theta)

**b. Linear classification, 2 classes, sigmoidal output** 

Here your classifier will act according to: `y_hat = sigmoid(wx)`. Make the same plots as in (a). In addition, plot the output of the sigmoid (i.e. `y_hat`) and interpret its values.

In [ ]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

def sigmoid_grad(z):
    return sigmoid(z)*(1 - sigmoid(z))

def compute_grad(X, y, theta):
    """Compute gradient"""
    m = len(X)
    theta_grad = np.zeros_like(theta)
    y_hat = sigmoid(X.dot(theta))
    for i in range(0, X.shape[1]):
        theta_grad[i] = -2*np.sum((y - y_hat)*sigmoid_grad(y_hat)*X[:, i])
    return theta_grad

In [ ]:
y = np.vstack([np.ones((100, 1)), np.zeros((100, 1))]).flatten()
X1 = np.hstack((np.random.normal(6, 2, size=(100,1)), np.random.normal(2, 2, size=(100, 1))))
X2 = np.hstack((np.random.normal(2, 3, size=(100,1)), np.random.normal(8, 2, size=(100, 1))))
X = np.vstack((X1, X2))

In [ ]:
plt.scatter(X[(y==1).flatten(), 0], X[(y==1).flatten(), 1], color='r')
plt.scatter(X[(y==0).flatten(), 0], X[(y==0).flatten(), 1], color='b')
plt.title('Generated data for problem 2b')
plt.show()

In [ ]:
m = len(X)
X = np.hstack((np.ones((m, 1)), X))

In [ ]:
# gradient descent
theta = np.array([1, 1, 1]) # intial theta (weight)
n_iter = 5000 # number of iteration
alpha = 0.05 # learning rate
for i in range(n_iter):
    theta_grad = compute_grad(X, y, theta)
    theta = theta - alpha*theta_grad

In [ ]:
y_hat = np.round(sigmoid(X.dot(theta))) # predicted class
print('Total correct classified = %s or %s percent' % (np.sum(y == y_hat), 100*np.sum(y == y_hat)/m))

**c. Linear classification, 3 classes, sigmoidal outputs**

Use the code in ‘ps1 3 classes sigmoid lda’ (3rd cell) to generate the data. Here there are three classes of data. There are three output units for the network–one for each class. When a data point is generated from the first class, the output is `[1 0 0];` from the second class, the output is `[0 1 0];` from the third class, the output is `[0 0 1]`. This is all in the variable y in the code. Your network should act according to `y_hat = sigmoid(Wx)`, where `y` is now a vector and `W` is a 3 by 3 matrix (3 dimensional output and 3 dimensional input). The predicted class for each data point

### **Classfier - back propagation** 

Your job is to create code implementing back propagation for a two layer neural network which can perform this classification. Use a network with 4 hidden units, as indicated in the shell code. Don’t worry about cross validation and all that – feel free to just use all the data.

## **Note for Matlab code to generate datasets for problem**

```
%% data for problem #2a

clear
dat1 = [normrnd(6,2,100,1) normrnd(2,1,100,1)];   % create the input data
dat2 = [normrnd(2,3,100,1) normrnd(8,1,100,1)];
dat = [dat1; dat2];

y = [ones(100,1); -1*ones(100,1)];   % these are the labels for the classes

% plot the data
ind1 = find(y == 1);
ind2 = find(y == -1);
plot(dat(ind1,1),dat(ind1,2),'r.')
hold on
plot(dat(ind2,1),dat(ind2,2),'b.')
hold off

%% data for problem #2b

clear
dat1 = [normrnd(6,2,100,1) normrnd(2,2,100,1)];  % create the input data
dat2 = [normrnd(2,3,100,1) normrnd(8,2,100,1)];
dat = [dat1; dat2];

y = [ones(100,1); 0*ones(100,1)];  % labels for classes

% plot the data
ind1 = find(y == 1);
ind2 = find(y == 0);
plot(dat(ind1,1),dat(ind1,2),'r.')
hold on
plot(dat(ind2,1),dat(ind2,2),'b.')
hold off

%% data for problem #2c

clear
dat1 = [normrnd(6,1,100,1) normrnd(2,1,100,1)];      % create the input data
dat2 = [normrnd(2,1,100,1) normrnd(8,1,100,1)];
dat3 = [normrnd(-2,1,100,1) normrnd(-2,1,100,1)];
dat = [dat1; dat2; dat3];

y(:,1) = [ones(100,1); zeros(100,1); zeros(100,1)];   % the class labels as three dimensional outputs
y(:,2) = [zeros(100,1); ones(100,1); zeros(100,1)]';
y(:,3) = [zeros(100,1); zeros(100,1); ones(100,1)]';
    
% plot the data
ind1 = find(y(:,1) == 1);
ind2 = find(y(:,2) == 1);
ind3 = find(y(:,3) == 1);
plot(dat(ind1,1),dat(ind1,2),'r.')
hold on
plot(dat(ind2,1),dat(ind2,2),'b.')
plot(dat(ind3,1),dat(ind3,2),'g.')
hold off

%% data for problem 3

clear
sd = .85;

x1 = [normrnd(0,sd,50,1); normrnd(0,sd,50,1);  normrnd(0,sd,50,1)];   % the input data
x2 = [normrnd(0,sd,50,1); normrnd(5,sd,50,1);   normrnd(10,sd,50,1)];
x3 = [ones(150,1)];
dat = [x1 x2 x3];
y = [ones(50,1) zeros(50,1) zeros(50,1);  zeros(50,1) ones(50,1) zeros(50,1); zeros(50,1) zeros(50,1) ones(50,1) ];  % the labels

% plot the data
ind1 = find(y(:,1) == 1);
ind2 = find(y(:,2) == 1);
ind3 = find(y(:,3) == 1);
plot(dat(ind1,1),dat(ind1,2),'r.')
hold on
plot(dat(ind2,1),dat(ind2,2),'b.')
plot(dat(ind3,1),dat(ind3,2),'g.')
hold off

% set up the starting parameters for the 2 layer network and weights
nsamp = length(x1);
ninput = 3;
nhidden = 4;
noutput = 3;

W = unifrnd(-1,1,ninput,nhidden);  % initialize weight matrices
V = unifrnd(-1,1,nhidden,noutput);

mu = .05; p = .9;   % a suggested step and momentum size

lastdW = 0*W;  lastdV = 0*V;   % initialize the previous weight change variables, to be used for the momentum

% now do back prop

```